# About

The script reads the Electricity Demand components from the ETYS Spatial CSV file created by running SAS script "Write_ETYS-Demands-CSV.sas". It processes it to produce the CSV files required for the visualisation map.

## Setup

In [42]:
#Import all libraries and set root driectory location
import shutil, os, json, pandas as pd
os.chdir("C:/Users/Rob.Nickerson/Desktop/regional-fes-resources-master")

In [162]:
#Delete the outputs folder and all it's content (to remove any old data).
dir_path = r".\ETYS data\Output"

output_folders = ["Active" , "DG", "DSR" ,"Sub1MW"]
            
try:
    shutil.rmtree(dir_path)
except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))

# We create a new empty Output folder.
try:
    os.mkdir(dir_path)
except OSError:
    print ("WARNING: Creation of the directory %s failed" % dir_path)
else:
    print ("Successfully created the directory %s " % dir_path)
    for items in output_folders:
        path = os.path.join(dir_path, items)
        os.mkdir(path)

Error: .\ETYS data\Output : The process cannot access the file because it is being used by another process


## Get list of regions in the spatail visualisation

In [163]:
#Get the list of Regions in the spatial visualisation
with open(r".\Geographies\GSPs 2019\GSP_post.geojson") as f:
    data = json.load(f) 

fin = open("./ETYS data/GSPs_VisualisationList.csv", "wt")
fin.write("Region")
fin.write('\n')

for feature in data['features']:
    fin.write(feature['properties']['GSP(s)'])
    fin.write('\n')
    
fin.close()

#Read resultant CSV in as GSP_Regions dataframe
GSP_regions = pd.read_csv(r".\ETYS data\GSPs_VisualisationList.csv")

GSP_regions

,Region
0,ABHA1
1,ABNE_P
2,ABTH_1
3,ACTL_2;CBNK_H;GREE_H;PERI_H
4,ALNE_P
...,...
312,WOHI_P
313,WTHU31
314,WWEY_1
315,WYLF_1


In [164]:
#explode GSP_regions into grouped_regions

grouped_regions = pd.concat([pd.Series(row['Region'], row['Region'].split(';'))              
                    for _, row in GSP_regions.iterrows()]).reset_index()    

grouped_regions

,index,0
0,ABHA1,ABHA1
1,ABNE_P,ABNE_P
2,ABTH_1,ABTH_1
3,ACTL_2,ACTL_2;CBNK_H;GREE_H;PERI_H
4,CBNK_H,ACTL_2;CBNK_H;GREE_H;PERI_H
...,...,...
354,WOHI_P,WOHI_P
355,WTHU31,WTHU31
356,WWEY_1,WWEY_1
357,WYLF_1,WYLF_1


## Read input data CSVs

We drop un-used columns as soon as possible. This prevents us from using them in the future without first considering what corrections need applying to them first.

In [165]:
pd.options.mode.chained_assignment = None  # default='warn'

pd.options.mode.chained_assignment = "warn"

# Read in the CSVs to dataframes
df_active_csv = pd.read_csv(r".\ETYS data\Input\active.csv")
df_active_csv = df_active_csv.drop(columns=['DemandAM', 'DemandPM'])
print(df_active_csv.head(10))

df_DG_csv = pd.read_csv(r".\ETYS data\Input\DG.csv")
df_DG_csv = df_DG_csv.drop(columns=['wintpk', 'summam', 'summpm'])
print(df_DG_csv.head(10))

df_Sub1MW_csv = pd.read_csv(r".\ETYS data\Input\Sub1MW.csv")
df_Sub1MW_csv = df_Sub1MW_csv.drop(columns=['wintpk', 'summam', 'summpm'])
print(df_Sub1MW_csv.head(10))

df_DSR_csv = pd.read_csv(r".\ETYS data\Input\DSR.csv")
print(df_DSR_csv.head(10))

  scenario    GSP  DemandPk type  year
0       SP  ABHA1    70.149    C    19
1       SP  ABHA1    70.912    C    20
2       SP  ABHA1    71.497    C    21
3       SP  ABHA1    71.974    C    22
4       SP  ABHA1    72.314    C    23
5       SP  ABHA1    72.688    C    24
6       SP  ABHA1    72.970    C    25
7       SP  ABHA1    73.172    C    26
8       SP  ABHA1    73.399    C    27
9       SP  ABHA1    73.573    C    28
  scenario   tech  year etys_location  capacity
0       CF  Hydro    20        ALNE_P      4.00
1       CF  Hydro    20        ARDK_P      5.20
2       CF  Hydro    20        ARMO_P      1.20
3       CF  Hydro    20        BEAU_P      1.99
4       CF  Hydro    20        BOAG_P      6.95
5       CF  Hydro    20        BRAC_P      3.00
6       CF  Hydro    20        BROA_P      1.20
7       CF  Hydro    20        CAAD_P      2.40
8       CF  Hydro    20          CAFA     12.00
9       CF  Hydro    20        CASS_P      4.50
  scenario     tech  year etys_location  ca

## Apply corrections
The ETYS spatial demand data has some edge case changes that we need to reverse out. These are:

- Several Scottish GSPs are split in the ETYS data. These are the "G_EXTRA" locations and we will need to set them back to their Elexon registered GSP for the data visualisation.
- There are cases of new GSPs added in the future taking a proportion of demand from other (existing) GSPs. As our visualisation does not show future new GSPs, we need to reverse the logic and add the demands back to their present GSP.

In [166]:
#Active
df_active = df_active_csv.copy()
df_active.loc[(df_active.GSP == 'G_EXTRA_1'),'GSP']='DUMF'
df_active.loc[(df_active.GSP == 'G_EXTRA_2'),'GSP']='DUMF'
df_active.loc[(df_active.GSP == 'G_EXTRA_3'),'GSP']='DUMF'
df_active.loc[(df_active.GSP == 'G_EXTRA_4'),'GSP']='GRMO'
df_active.loc[(df_active.GSP == 'G_EXTRA_5'),'GSP']='GRMO'
df_active.loc[(df_active.GSP == 'G_EXTRA_6'),'GSP']='KILB'
df_active.loc[(df_active.GSP == 'G_EXTRA_7'),'GSP']='KILB'
df_active.loc[(df_active.GSP == 'G_EXTRA_8'),'GSP']='SACO'
df_active.loc[(df_active.GSP == 'G_EXTRA_9'),'GSP']='SACO'
df_active.loc[(df_active.GSP == 'G_EXTRA_10'),'GSP']='CROO'
df_active.loc[(df_active.GSP == 'G_EXTRA_11'),'GSP']='CROO'
df_active.loc[(df_active.GSP == 'DUNB_A'),'GSP']='DUNB'
df_active.loc[(df_active.GSP == 'DUNB_B'),'GSP']='DUNB'

##Reverse out G_EXTRA_13
df_active.loc[(df_active.GSP == 'KINT_P') & (df_active.year >= 21),'DemandPk'] = df_active.DemandPk / (1 - 0.1164)
df_active.loc[(df_active.GSP == 'KEIT_P') & (df_active.year >= 21),'DemandPk'] = df_active.DemandPk / (1 - 0.14457)

##Reverse out G_EXTRA_14
#Nothing to do for FES 2021.

##Reverse out G_EXTRA_15
df_active.loc[(df_active.GSP == 'LINM') & (df_active.year >= 25),'DemandPk'] = df_active.DemandPk / (1 - 0.13)

##Reverse out G_EXTRA_16
df_active.loc[(df_active.GSP == 'CHAP') & (df_active.year >= 23),'DemandPk'] = df_active.DemandPk / (1 - 0.42)

##Reverse out ISLI_1
#Commented out as we have actually added ISLI_1 to the geojson making a (approx) assumption that it is splitting only from WHAM_1.
#df_active.loc[(df_active.GSP == 'LODR_6') & (df_active.year >= 22),'DemandPk'] = df_active.DemandPk / (1 - 0.25)
#df_active.loc[(df_active.GSP == 'WHAM_1') & (df_active.year >= 22),'DemandPk'] = df_active.DemandPk / (1 - 0.45)


In [167]:
#DG
df_DG = df_DG_csv.copy()
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_1'),'etys_location']='DUMF'
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_2'),'etys_location']='DUMF'
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_3'),'etys_location']='DUMF'
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_4'),'etys_location']='GRMO'
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_5'),'etys_location']='GRMO'
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_6'),'etys_location']='KILB'
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_7'),'etys_location']='KILB'
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_8'),'etys_location']='SACO'
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_9'),'etys_location']='SACO'
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_10'),'etys_location']='CROO'
df_DG.loc[(df_DG.etys_location == 'G_EXTRA_11'),'etys_location']='CROO'
df_DG.loc[(df_DG.etys_location == 'DUNB_A'),'etys_location']='DUNB'
df_DG.loc[(df_DG.etys_location == 'DUNB_B'),'etys_location']='DUNB'

##Reverse out others
#We don't have the ability to do that at this point in the analysis as we have aggregated technology types together.
#TODO: Look at our upstream processes and change them so that we produce the visualisation data at that stage.


In [168]:
#Sub1MW
df_Sub1MW = df_Sub1MW_csv.copy()
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_1'),'etys_location']='DUMF'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_2'),'etys_location']='DUMF'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_3'),'etys_location']='DUMF'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_4'),'etys_location']='GRMO'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_5'),'etys_location']='GRMO'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_6'),'etys_location']='KILB'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_7'),'etys_location']='KILB'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_8'),'etys_location']='SACO'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_9'),'etys_location']='SACO'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_10'),'etys_location']='CROO'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'G_EXTRA_11'),'etys_location']='CROO'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'DUNB_A'),'etys_location']='DUNB'
df_Sub1MW.loc[(df_Sub1MW.etys_location == 'DUNB_B'),'etys_location']='DUNB'

##Reverse out others
#We don't have the ability to do that at this point in the analysis as we have aggregated technology types together.
#TODO: Look at our upstream processes and change them so that we produce the visualisation data at that stage.


In [169]:
#DSR
df_DSR = df_DSR_csv.copy()
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_1'),'GSP']='DUMF'
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_2'),'GSP']='DUMF'
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_3'),'GSP']='DUMF'
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_4'),'GSP']='GRMO'
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_5'),'GSP']='GRMO'
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_6'),'GSP']='KILB'
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_7'),'GSP']='KILB'
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_8'),'GSP']='SACO'
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_9'),'GSP']='SACO'
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_10'),'GSP']='CROO'
df_DSR.loc[(df_DSR.GSP == 'G_EXTRA_11'),'GSP']='CROO'
df_DSR.loc[(df_DSR.GSP == 'DUNB_A'),'GSP']='DUNB'
df_DSR.loc[(df_DSR.GSP == 'DUNB_B'),'GSP']='DUNB'

##Reverse out others
#TODO.


## Produce outputs

In [170]:
#Active.csv

for category in ["C", "D", "E", "H", "I", "R", "Z"]:
    for scenario in ["SP", "CT", "ST", "LW"]:
        # Create df as a filter of df_csv
        df = df_active[(df_active['scenario'] == scenario) & (df_active['type'] == category)]
        df['year'] = df['year'] + 2000
        
        #Merge df on grouped_regions. Drop index & GSP columns
        df = df.merge(grouped_regions, left_on = "GSP", right_on = "index", how = 'left').drop(columns = ["index" , "GSP"]).rename(columns = {0:"Region"})
        
        # Pivot to have years across the top
        df = pd.pivot_table(df, index='Region', columns= 'year', values='DemandPk', aggfunc = 'sum')
        
        # Export to CSV
        df.index.name = 'Primary'
        filename = scenario + "-DemandPk-" + category + ".csv"
        df.to_csv(r".\ETYS data\Output\Active\\" + filename, index=True, float_format='%.3f')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [171]:
#Active.csv (Aggregated)

for scenario in ["SP", "CT", "ST", "LW"]:
    # Create df as a filter of df_csv
    df = df_active[(df_active['scenario'] == scenario)]
    df['year'] = df['year'] + 2000

    #Merge df on grouped_regions. Drop index & GSP columns
    df = df.merge(grouped_regions, left_on = "GSP", right_on = "index", how = 'left').drop(columns = ["index" , "GSP"]).rename(columns = {0:"Region"})

    # Pivot to have years across the top
    df = pd.pivot_table(df, index='Region', columns= 'year', values='DemandPk', aggfunc = 'sum')

    # Export to CSV
    df.index.name = 'Primary'
    filename = scenario + "-DemandPk-" + "All" + ".csv"
    df.to_csv(r".\ETYS data\Output\Active\\" + filename, index=True, float_format='%.3f')


C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [172]:
#DG.csv

for technology in ["Hydro", "Other", "Solar", "Storage", "Wind"]:
    for scenario in ["SP", "CT", "ST", "LW"]:
        # Create df as a filter of df_csv
        df = df_DG[(df_DG['scenario'] == scenario) & (df_DG['tech'] == technology)]
        df['year'] = df['year'] + 2000
           
        #Merge df on grouped_regions. Drop index & etys_location columns
        df = df.merge(grouped_regions, left_on = "etys_location", right_on = "index", how = 'left').drop(columns = ["index" , "etys_location"]).rename(columns = {0:"Region"})
        
        # Pivot to have years across the top
        df = pd.pivot_table(df, index='Region', columns= 'year', values='capacity', aggfunc = 'sum')
        
        # Export to CSV
        df.index.name = 'Primary'
        filename = scenario + "-DxCapacity-" + technology + ".csv"
        df.to_csv(r".\ETYS data\Output\DG\\" + filename, index=True, float_format='%.3f')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [177]:
#Sub1MW.csv

for technology in ["Hydro", "Other", "Solar", "Battery", "Wind"]:
    for scenario in ["SP", "CT", "ST", "LW"]:
        # Create df as a filter of df_csv
        df = df_Sub1MW[(df_Sub1MW['scenario'] == scenario) & (df_Sub1MW['tech'] == technology)]
        df['year'] = df['year'] + 2000
        
        #Merge df on grouped_regions. Drop index & GSP columns
        df = df.merge(grouped_regions, left_on = "etys_location", right_on = "index", how = 'left').drop(columns = ["index" , "etys_location"]).rename(columns = {0:"Region"})
        
        # Pivot to have years across the top
        df = pd.pivot_table(df, index='Region', columns= 'year', values='capacity', aggfunc = 'sum')
        
        # Export to CSV
        df.index.name = 'Primary'
        if technology == "Battery":
            filename = scenario + "-MxCapacity-" + "Storage" + ".csv"
        else:
            filename = scenario + "-MxCapacity-" + technology + ".csv"
        df.to_csv(r".\ETYS data\Output\Sub1MW\\" + filename, index=True, float_format='%.3f')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [174]:
#DSR.csv

for scenario in ["SP", "CT", "ST", "LW"]:
    # Create df as a filter of df_csv
    df = df_DSR[(df_DSR['scenario'] == scenario)]
    df['year'] = df['year'] + 2000

    #Merge df on grouped_regions. Drop index & GSP columns
    df = df.merge(grouped_regions, left_on = "GSP", right_on = "index", how = 'left').drop(columns = ["index" , "GSP"]).rename(columns = {0:"Region"})
    
    # Pivot to have years across the top
    df = pd.pivot_table(df, index='Region', columns= 'year', values='DSR', aggfunc = 'sum')

    # Export to CSV
    df.index.name = 'Primary'
    filename = scenario + "-DSR-" + ".csv"
    df.to_csv(r".\ETYS data\Output\DSR\\" + filename, index=True, float_format='%.3f')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# QA

Note: The active dataset also includes transmission sites that we do not include in the Regional Visulisation. These start "T_", "M_" "B_EXTRA".